# Morphological analysis

- create list of 50 signs and their synonyms from the course
- find a way to group two words into one sign
- read about Morphological analysis to make the pairs to solve the problem of verbs & nouns
- find way to apply rules like removing  على
- find difference between كتب which is book and wrote 




# very important

- كلمة بيتك وبيتى او اى كلمة فيها ملكية ف وقتها حولها للكلمة الاصلية  اللى هيا هنا هتكون بيت وبعدين اعمل اشارة ملكية او اشارة للمخاطب اللى هيا بتكون انك بتشاور على نفسك او بتشاور امامك وايدك مقفولة
- المذكر والمؤنث فى كلمات عديدة يتم باستعمال اشارة الكلمة نفسها تليها اشارة لتحديد المذكر او المؤنث
- some words don't follow the previous rule so i have to seperate between the words that follow these rules and the words that don't
- اشارة اخت عبارة عن اشارة اخ + اشارة امرأة ونفس الامر لكمات عديدة
- فكرة الملكية بنتطبق برضو على الجمع انة اشارة الكلمة وبعدين جمع او لا 
- مش بحب بتتحول ل بحب لا  ودة بينطبق على كلمات كثير ف حاول تشوف حل لانك تعكس الترتيب دة 
- بعض الاوقات كلمة الجمع مش بتتحط عشان مش مهمة اوى ف حتى لو الكلمة جمع انت بتعمل اشارة المفرد من غير ما تحط حاجة تبين ان قصدك الجمع ودة بيتفهم من السياق
- بعض الاوقات الازمنة مهمة واوقات تانى لا ف مثلا شرب و بشرب وهشرب ممكن يترجموا لنفس الاشارة فى الاخر او ممكن يترجموا ل شرب ماضى او شرب الان او شرب مستقبل واللى بيكون اشارة بتعبر عن الحاجة ووراها اشارة بتعبر عن الزمن
- فى لغة الاشارة بتعرف الفاعل من الفعل ان الفاعل بيكون عبارة عن اشارة شخص + اشارة الفعل نفسة

# confusing words so far
-  غيرة it could me jealousy or some on else
- قلب heart or middle ex : فى قلب الاحداث او قلب انسان
- علم flag or science
- كتب wrote or books
- مركز focus or position

# 1 - load words

In [1]:
import os,json,re

In [25]:
directory = "signLanguageData"
word_dic = {}
letters_dic = {}

for file_name in os.listdir(directory):
    path = os.path.join(directory,file_name)
    with open(path, 'r') as json_file:
        json_data = json.load(json_file)
        for sign,value in json_data.items():
            for word in value['words']:
                word_dic[word] = sign

with open("signLanguageLetters.json", 'r') as json_file:
    json_data = json.load(json_file)
    for sign,letter in json_data.items():
        letters_dic[letter] = sign
                
print(len(word_dic))
print(len(letters_dic)) # 29 letter is + used as و 

995
32


In [26]:
print("سقف" in word_dic)
print("سكينة" in word_dic)
print("خنزير" in word_dic)
print("شماعة" in word_dic)
print("طبيب" in word_dic)
print("دكتور" in word_dic)
print("بطريق" in word_dic)

True
True
True
True
True
True
False


In [27]:
# view two words with the same sign (synonyms)
print(word_dic["شماعة"] )
print(word_dic["طبيب"] )
print(word_dic["دكتور"] ) # طبيب هى نفس معنى دكتور لذلك لهم نفس الاشارة

sign_002_33
sign_007_7
sign_007_7


# 2 - load sentence

In [28]:
name = "sentences.json"


sign_language_data = os.path.join("sentences.json")

sentence_list = None
with open(sign_language_data, 'r') as json_file:
    sentence_list = json.load(json_file)

    

    
def compare(a,b):
    if(len(a)!=len(b)):
        return False
    
    for i,j in zip(a,b):
        if i!=j:
            return False
    
    return True
    
def run_tests(func,json_data):
    errors = []
    successes = []
    for data in json_data:
        output = func(data['input'])
        cmp = compare(output,data['output'])
        if(cmp==1):
            successes.append({
                "input":data['input'],
                "output":data['output'],
            })
        else:
            errors.append({
                "input":data['input'],
                "output":data['output'],
                "my_output":output
            })
    
    if len(errors)>0:
        print("successfull ",len(successes),"of",len(json_data))
        for success in successes:
            print(success)
        print("fail",len(errors))
        for error in errors:
            print(error)
        
    else:
        print("all successfull ",len(successes),"of",len(successes))
        



In [29]:
def test(inp):
    return ["انت","اسم","ملكك","اية"]

# first parameter is the function to apply the input on , second parameter is the input string (list of strings)
run_tests(test,sentence_list[:5])

successfull  1 of 5
{'input': 'انت اسمك اية', 'output': ['انت', 'اسم', 'ملكك', 'اية']}
fail 4
{'input': 'السلام عليكم ورحمة الله وبركاتة', 'output': ['سلام', 'عليكم', 'رحمة', 'بركاتة'], 'my_output': ['انت', 'اسم', 'ملكك', 'اية']}
{'input': 'اسمك اية', 'output': ['اسم', 'ملكك', 'اية'], 'my_output': ['انت', 'اسم', 'ملكك', 'اية']}
{'input': 'كيف حالك', 'output': ['كيف', 'حال'], 'my_output': ['انت', 'اسم', 'ملكك', 'اية']}
{'input': 'انت عامل اية ', 'output': ['انت', 'عامل', 'اية'], 'my_output': ['انت', 'اسم', 'ملكك', 'اية']}


In [30]:
word_dic['حالك']

'sign_001_7'

# 3 - text cleaning

### 3.1 split by words

In [31]:
# convert string to list of words
def get_list(text):
    # remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = text.replace("\n"," ")
    text = re.sub(p_longation, subst, text)
    text = text.strip().replace("  "," ")
    
    return text.split(" ")

In [32]:
s = """
صباااااااح الخير
كيف حالك
انا احب الشمس  والقمر
"""
print(get_list("صبااااااح الخير\n ازاى انت عايش"))
print(get_list(s))

['صبااح', 'الخير', 'ازاى', 'انت', 'عايش']
['صبااح', 'الخير', 'كيف', 'حالك', 'انا', 'احب', 'الشمس', 'والقمر']


### 3.2 - remove or replace specific letters

In [33]:
# remove repeated letters ا & ى & و
# other transformations

replacing = {
#     "ة":"ه", #  convert ه to ة
    "أ":"ا",
    "آ":"ا",
    "اّ":"ا",
    "إ":"ا",
    "ى":"ي"
}


def replace_letters(text):
    for key,value in replacing.items():
        text = text.replace(key,value)
    
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    return text


# remove ال
def clean_str(text):
    # convert list to string to apply the next operations
    text = " ".join(text)
    
    text = replace_letters(text)
    
    text = text.split(" ")
    
    # add this processing into other place
    final = []
    for i,word in enumerate(text):
        final.append(word)
    
    return final
    
    

# 4 - root extraction

#### problems
- احزانى is converted worng
- يكرهون should only be يكرة without adding ون because it's a verb not a noun  
- fix the probelm of ة and ه 
- fix the problem of والقمر to be able to become و القمر

In [34]:
class ArabicRoot:
    def __init__(self,word_list):
        self.word_list = word_list
        self.plural = ['ات',"ون","ين","ان"]
        self.mine = ['ني',"تي"]
        

        self.pre6 = ['كال',"بال","فال","مال","ولل","است","يست","تست","مست","وال"]
        self.pre5 = ["سن","ست","سى","لي","لن","لت","لل"]
        self.pre4 = ["ت","ي","ب","ل"]



        self.suf5 = ["ون","ات","ان","ين","تن","تم","كن","كم","هن","هم","يا","ني","تي","وا","ما","نا","ية","ها","اء"]
        self.suf4 = ['ت',"ة","ا","ي"]
    
    def remove_prefix(self,word):
        if(len(word)>=6):
            for pre in self.pre6:
                if word.startswith(pre):
                    return word[3:]
        if(len(word)>=5):
            for pre in self.pre5:
                if word.startswith(pre):
                    return word[2:]
        if(len(word)>=4):
            for pre in self.pre4:
                if word.startswith(pre):
                    return word[1:]
        
        return word
                
    def get_extra_words(self,word):
        if(len(word)>=5):                
            for suf in self.plural:
                if word.endswith(suf) and word[:-2] in self.word_list:
                    return [word[:-2],"كثير"]
                
        if(word[-1]=='ي' and word[:-1] in self.word_list):
            return [word[:-1],"ملكى"]
        if(word[-1]=='ك' and word[:-1] in self.word_list):
            return [word[:-1],"ملكك"]
        if(word[-1]=='ة' and word[:-1] in self.word_list):
            return [word[:-1],"مؤنث"]
        return []
    
    def remove_suffix(self,word):        
        if(len(word)>=5):                
            for suf in self.suf5:
                if word.endswith(suf):
                    return word[:-2]
        if(len(word)>=4):
            for suf in self.suf4:
                if word.endswith(suf):
                    return word[:-1]
        
        return word

    def clean_word(self,word):
        return self.remove_suffix(self.remove_prefix(word))
    
    def remove_suffix_final(self,word):
        output = self.get_extra_words(word)
        return output if len(output)>0 else self.remove_suffix(word)
        

In [38]:
obj = ArabicRoot(word_dic)
synonyms_dic = {
    "ازيك" : ["كيف","حالك"],
    "و":"+",
}

def get_synonyms(lis):
    final = []
    for word in lis:
        if word in synonyms_dic:
            final.extend(synonyms_dic[word])
        else:
            final.append(word)
    return final

def translate(lis):
    final = []
    for word in lis:
        if word in word_dic:
            final.append(word)
            continue
        
        if(len(word)>3 and word[:2]=="ال"):
            word = word[2:]
            if word in word_dic:
                final.append(word)
                continue
            
        
        if(len(word)>=6 and word[:3]=="وال"):
            word = word[3:]
            final.append('+')
            if word in word_dic:
                final.append(word)
                continue
        
        output = obj.get_extra_words(word)
        if(len(output)>0):
            final.extend(output)
            continue
        
        final_form = obj.clean_word(word)
        if(final_form in word_dic):
            final.append(word)
            continue
        
        
        
        if(len(final)>1 and (len(final[-1])==1 and final[-1]!='+' and word!='+') ):
            final.append(" ")
        for letter in word:
            final.append(letter)
        
    return final
    
    
    



def get_signs(lis):
    final =[]
    for word in lis:
        if len(word)>1:
            final.append(word_dic[word])
        else:
            final.append(letters_dic[word])
    return final



def pipeline(s):
    lis = get_list(s)
    print(lis)
    
    cleaned = clean_str(lis)
    print(cleaned)
    
    synonyms = get_synonyms(cleaned)
    print(synonyms)
    
    translated = translate(synonyms)
    print(translated)
    
    signs = get_signs(translated)    
    for sign in signs:
        print(sign)
    
    return signs


In [41]:
s =[ """
صباااااااح الخير
كيف حالك
انا احب الشمس  والقمر
""",
"كيف حالك",
"قال الخنزير اليوم ملكي ",
"الطالب يكرة المدرسة لكن يحب الجامعة",
"ازيك يا باشا عامل اية",
"الحيوانات توجد فى حديقة الحيوانات",
"المهندسون يكرهون الجامعة",
"الحمد لله"
    
]
for example in s:
    output = pipeline(example)
    print("\n")

# for sent in sentence_list[:10]:
#     s = sent['input']
#     pipeline(s)
#     print("\n\n")

['صبااح', 'الخير', 'كيف', 'حالك', 'انا', 'احب', 'الشمس', 'والقمر']
['صباح', 'الخير', 'كيف', 'حالك', 'انا', 'احب', 'الشمس', 'والقمر']
['صباح', 'الخير', 'كيف', 'حالك', 'انا', 'احب', 'الشمس', 'والقمر']
['صباح', 'الخير', 'كيف', 'حالك', 'انا', 'احب', 'ش', 'م', 'س', '+', 'ق', 'م', 'ر']
sign_012_1
sign_001_13
sign_001_8
sign_001_7
sign_001_19
sign_001_70
sign_000_13
sign_000_24
sign_000_12
sign_000_31
sign_000_21
sign_000_24
sign_000_10


['كيف', 'حالك']
['كيف', 'حالك']
['كيف', 'حالك']
['كيف', 'حالك']
sign_001_8
sign_001_7


['قال', 'الخنزير', 'اليوم', 'ملكي']
['قال', 'الخنزير', 'اليوم', 'ملكي']
['قال', 'الخنزير', 'اليوم', 'ملكي']
['قال', 'خنزير', 'يوم', 'ملكي']
sign_005_90
sign_013_2
sign_012_9
sign_001_21


['الطالب', 'يكرة', 'المدرسة', 'لكن', 'يحب', 'الجامعة']
['الطالب', 'يكرة', 'المدرسة', 'لكن', 'يحب', 'الجامعة']
['الطالب', 'يكرة', 'المدرسة', 'لكن', 'يحب', 'الجامعة']
['طالب', 'ي', 'ك', 'ر', 'ة', 'مدرسة', 'ل', 'ك', 'ن', 'يحب', 'جامعة']
sign_007_16
sign_000_28
sign_000_22
sign_000_10
sign_0

In [318]:
obj.get_extra_words('يكرهون')

[]

# 5 - multiple words (grouping)

In [37]:
import os
import json

sign_language_words = os.path.join("signLanguageWords","تعارف.txt")
sign_language_data = os.path.join("signLanguageData","تعارف.json")

json_data = {}
existing_words = {}
counter = 0;

with open(sign_language_data, 'r') as json_file:
    json_data = json.load(json_file)
    counter = len(json_data)
    for key,value in json_data.items():
        existing_words[value['word']] = 1;


    
with open(sign_language_words,"r") as f:
    lis = f.read().split("\n")
    
    for word in lis:
        if(word not in existing_words):
            json_data[f'sign_{counter}'] = {"word":word}
            counter+=1
    with open(sign_language_data, 'w') as json_file:
        json.dump(json_data,json_file)




for key,value in json_data.items():
    print(key,value['word'])

sign_0 تعارف
sign_1 السلام
sign_2 عليكم
sign_3 رحمه
sign_4 الله
sign_5 بركاته


In [16]:
def translate(arr):
    final = []
    for word in arr:
        if word in synonyms_table:
            final.append(synonyms_table[word])
        else:
            for letter in word:
                final.append(letter)
    return final

In [18]:
# ex ; we don't have سار but we  have ذهب 
# we don't have طريق so it spelled it 
text = clean_str("السلام عليكم ورحمه الله وبركاته")
results = lemmatization(text)
translate(results)

['س',
 'ل',
 'ا',
 'م',
 'ع',
 'ل',
 'ى',
 'ر',
 'ح',
 'م',
 'ا',
 'ل',
 'ل',
 'ه',
 'ب',
 'ر',
 'ك',
 'ة']

# steaming (might be used in the future)

In [162]:
import nltk

In [163]:
# https://www.nltk.org/api/nltk.stem.isri.html
obj = nltk.stem.isri.ISRIStemmer()


In [29]:
for i in ["الخير"]:
    print(obj.stem(i))

خير


In [17]:
for i in ["الاستنشاق","المحبة","الرحمة","يذهب","السما"]:
    print(obj.stem(i))

نشق
حبة
رحم
ذهب
سما


In [165]:
obj.suf32("احزان")

'احز'

In [112]:
obj.stem("اشتغفار")

'شتغفار'